# 第一章

## The parallel computing memory architecture
* Single instruction, single data (SISD)
![](./figs/SISD.png)
* Single instruction, multiple data (SIMD)

* Multiple instruction, single data (MISD)
![](./figs/MISD.png)
* Multiple instruction, multiple data (MIMD)
![](./figs/MIMD.png)

## Memory organization
![](figs/memory_MIMD.png)

## Shared memory
![](./figs/share_memory.png)

## Distributed memory
![](./figs/Distributed_memory.png)

## Parallel programming models
* The shared memory model
* The multithread model
* The distributed memory/message passing model
* The data parallel model

### The shared memory model
在此模型中，任务共享一个共享内存区域，其中对共享资源的访问（读取和写入数据）是异步的。 有一些机制允许程序员控制对共享内存的访问，例如，锁或信号灯。 这种模型的优点是程序员不必明确任务之间的通信。 在性能方面的一个重要缺点是，它变得更加难以理解和管理数据局部性。 将数据保留在使用该处理器的处理器本地，可以节省内存访问，高速缓存刷新以及在多个处理器使用同一数据时发生的总线流量。

### The multithread model

### The message passing model
消息传递模型通常适用于每个处理器都有自己的内存（分布式内存系统）的情况。 更多任务可以驻留在同一台物理计算机上或任意数量的计算机上。 程序员负责确定通过消息进行的并行性和数据交换。 此并行编程模型的实现需要使用要使用的（临时）软件库
在代码中。 线程/进程通信对应着消息传递模式.

### The data parallel model
![](./figs/data_parallel.png)

## How to evaluate the performance of a parallel program


### Speedup

$S = \frac{T_{S}}{T_{p}}$, p identical processing elements

* S = p is linear or ideal speedup
* S < p is real speedup
* S > p is superlinear speedup

## Start working with processes in Python

In [1]:
# import os
# import sys
# ##this is the code to execute
# program = "python"
# print("Process calling")
# arguments = ["called_Process.py"]
# ##we call the called_Process.py script
# os.execvp(program, (program,) + tuple(arguments))
# print("Good Bye!!")

## Start working with threads in Python

* 基于线程的并行性是编写并行程序的标准方法。 但是，Python解释器不是完全线程安全的。 为了支持多线程Python程序，使用了称为全局解释器锁（GIL）的全局锁。 这意味着只有一个线程可以同时执行Python代码。 短时间或某个线程执行可能需要一段时间的操作后，Python会自动切换到下一个线程。
* 使用线程时要注意的一个关键点是，必须始终确保永远不要让任何线程在后台运行。

In [2]:
from threading import Thread
##Also we use the sleep function to make the thread "sleep"
from time import sleep
## To create a thread in Python you'll want to make your class work as a thread.
## For this, you should subclass your class from the Thread class
class CookBook(Thread):
    def __init__(self):
        Thread.__init__(self)
        self.message = "Hello Parallel Python CookBook!!\n"
    ##this method prints only the message
    def print_message(self):
        print (self.message)
        ##The run method prints ten times the message
    def run(self):
        print ("Thread Starting\n")
        x=0
        while (x < 10):
            self.print_message()
            sleep(2)
            x += 1
        print ("Thread Ended\n")
#start the main process
print ("Process Started")

# create an instance of the HelloWorld class
hello_Python = CookBook()
# print the message...starting the thread
hello_Python.start()
#end the main process
print ("Process Ended")

Process Started
Thread Starting

Hello Parallel Python CookBook!!

Process Ended


## 并发例子

In [3]:
import multiprocessing

def make_data(queue, num, work_nums):
    for i in range(num):
        queue.put(i)
    for i in range(work_nums):
        queue.put(None)

def handle_data(queue, share_value, lock):
    while True:
        data = queue.get()
        if data is None:
            break
        lock.acquire()
        share_value.value = share_value.value + data
        lock.release()

if __name__ == "__main__":
    queue = multiprocessing.Queue()  # 进程间通信所用
    share_value = multiprocessing.Value("i", 0)  # 进程间共享所用
    lock = multiprocessing.Lock()  # 进程间共享内存时，采用锁同步机制
    num = 10000  #
    work_nums = 5  # work进程个数
    sub_process = []  # 处理数据进程集合

    master_process = multiprocessing.Process(target=make_data, args=(queue, num, work_nums, ))  # 生成数据进程
    for i in range(work_nums):
        sub_process1 = multiprocessing.Process(target=handle_data, args=(queue, share_value, lock,))
        sub_process.append(sub_process1)

    master_process.start()
    for p in sub_process:
        p.start()

    master_process.join()
    for p in sub_process:
        p.join()

    # 结果对比
    result = 0
    for i in range(num):
        result = result + i
    print("result should be " + str(result))
    print("fact is " + str(share_value.value))

result should be 49995000
fact is 49995000


## 并发(concurrency) vs 并行(parallellism)
* Concurrency is not Parallelism
* Concurrency enables parallelism & makes parallelism (and scaling and everything else) easy

* 当有多个线程在操作时，如果系统只有一个 CPU，则它根本不可能真正同时进行一个以上的线程，它只能把 CPU 运行时间划分成若干个时间段，再将时间段分配给各个线程执行，在一个时间段的线程代码运行时,其它线程处于挂起状态.这种方式我们称之为并发（Concurrent）。

* 当系统有一个以上 CPU 时，则线程的操作有可能非并发。当一个 CPU 执行一个线程时，另一个 CPU 可以执行另一个线程，两个线程互不抢占 CPU 资源，可以同时进行，这种方式我们称之为并行（Parallel）

![](./figs/concurrent_parallel.png)

## 阻塞与非阻塞:

* 阻塞是指调用线程或者进程被操作系统挂起。
* 非阻塞是指调用线程或者进程不会被操作系统挂起。

## 同步与异步:
* 同步是阻塞模式: 同步就是指一个进程在执行某个请求的时候，若该请求需要一段时间才能返回信息，那么这个进程将会一直等待下去，知道收到返回信息才继续执行下去；

* 异步是非阻塞模式:异步是指进程不需要一直等下去，而是继续执行下面的操作，不管其他进程的状态。当有消息返回式系统会通知进程进行处理，这样可以提高执行的效率。


## 协程
* 协程，又称微线程，纤程。英文名Coroutine。一句话说明什么是线程：协程是一种用户态的轻量级线程。协程不是被操作系统内核所管理，而完全是由程序所控制

协程的好处：
* 无需线程上下文切换的开销
* 无需原子操作锁定及同步的开销
* 方便切换控制流，简化编程模型

缺点：
* 无法利用多核资源：协程的本质是个单线程,它不能同时将 单个CPU 的多个核用上,协程需要和进程配合才能运行在多CPU上.当然我们日常所编写的绝大部分应用都没有这个必要，除非是cpu密集型应用。
* 进行阻塞（Blocking）操作（如IO时）会阻塞掉整个程序


In [4]:
# python3.7 需要安装pip install tornado==4.5.3
import asyncio,time

@asyncio.coroutine #设为异步函数
def func1(num):
    print(num,'before---func1----')
    yield from asyncio.sleep(5)
    print(num,'after---func1----')

task = [func1(1),func1(2)]

if __name__ == "__main__":
    begin = time.time()
    loop = asyncio.get_event_loop() #进入事件循环
    loop.run_until_complete(asyncio.gather(*task)) #将协同程序注册到事件循环中
    loop.close()
    end = time.time()
    print(end-begin)

1 before---func1----
2 before---func1----
Hello Parallel Python CookBook!!

Hello Parallel Python CookBook!!

1 after---func1----
2 after---func1----
5.007609128952026


In [5]:
from greenlet import greenlet

def foo():
    print("foo")
    bar()

def bar():
    a = 3 + 1
    print(a)
    gr2.switch()
    print("end bar")


gr1 = greenlet(bar)
gr2 = greenlet(foo)
gr1.switch()

4
foo
4
end bar
Hello Parallel Python CookBook!!

Hello Parallel Python CookBook!!

Hello Parallel Python CookBook!!

Hello Parallel Python CookBook!!

Hello Parallel Python CookBook!!

Hello Parallel Python CookBook!!

Hello Parallel Python CookBook!!

Thread Ended

